In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import pickle
from sklearn.model_selection import GridSearchCV
import warnings
import math

KeyboardInterrupt: 

In [ ]:
x1=pd.read_csv("x_m1.csv")
x_test1=pd.read_csv("x_test.csv")
y1=pd.read_csv("y_m1.csv")
y_test1=pd.read_csv("y_test.csv")

In [ ]:
XM1=pd.read_csv("Data - M1.csv")
YM1=XM1.TAXI_OUT
XM1=XM1.drop(["TAXI_OUT","cond"],axis=1)
XM2=pd.read_csv("Data - M2.csv")
YM2=XM2.j_del
XM2=XM2.drop(["arr_del","j_del","cond"],axis=1)

In [ ]:
#num_basis addition
category_list=['wind']
#cat_var=['MONTH','DAY_OF_WEEK','OP_UNIQUE_CARRIER',]
cat_var=[]

In [ ]:
def change(a):
    if a>=20:
        a=a+1
    if a>=10:
        a=a+2
    return a

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
for s in category_list:
    XM1[s] = lb_make.fit_transform(XM1[s])
    print(lb_make.transform(lb_make.classes_))
    print(lb_make.classes_)
    XM2[s] = lb_make.fit_transform(XM2[s])
    print(lb_make.transform(lb_make.classes_))
    print(lb_make.classes_)

print(XM1.shape)

In [ ]:
XM1.head(4),XM2.head(4)

In [ ]:
XM3=XM2[YM2>=0]
YM3=YM2[YM2>=0]

In [ ]:
ind=XM3.index.values

In [ ]:
def v(a):
    if a<5:return 0
    elif a<10: return 1
    elif a<15: return 2
    elif a<20: return 3
    elif a<30: return 4
    else: return 5
def v2(a):
    if a<0: return 0
    else: return 1
YM3=YM3.apply(v)
YM2=YM2.apply(v2)

In [ ]:
from lightgbm import LGBMRegressor
import lightgbm as lgb
lgbm = LGBMRegressor()
model1= lgb.Booster(model_file=r'./taxi_light.txt')

In [ ]:
import shap

# print the JS visualization code to the notebook
shap.initjs()
modelt=model1
modelt.params["objective"] = "regression"
explainer1= shap.TreeExplainer(modelt)
modelt.attr('objective')

In [ ]:
explainer1.expected_value

In [ ]:
pred1=model1.predict(XM1)
pred1.shape

In [ ]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(YM1,pred1),"mae")
from sklearn.metrics import median_absolute_error
print(median_absolute_error(YM1,pred1),"med a e")


In [ ]:
x_train=pd.read_csv("x_train3.csv")
x_test=pd.read_csv("x_test3.csv")
y_train=pd.read_csv("y_train3.csv")
y_test=pd.read_csv("y_test3.csv")

In [ ]:
from lightgbm import LGBMClassifier
import lightgbm as lgb
lgbm = LGBMClassifier()
model3= lgb.Booster(model_file=r'./arr_light_all_class.txt')
import shap

In [ ]:
model2= lgb.Booster(model_file=r'./arr_light_one_class.txt')

In [ ]:
from sklearn.metrics import classification_report
res=[np.round(i) for i in model2.predict(XM2)]
print(np.array(np.unique(res, return_counts=True)).T)
#print(y_test_m2)
print(classification_report(YM2,res))

In [ ]:
from sklearn.metrics import classification_report
res=[np.argmax(i) for i in model3.predict(XM3)]
print(res)
print(np.array(np.unique(res, return_counts=True)).T)
#print(y_test_m2)
print(classification_report(YM3,res))

In [ ]:
YM3.head(65).tail(5)

In [ ]:
#Converting to our way of checking
cls=[0]*len(YM3)
cls2=[0]*len(YM3)
for i in range(0,len(YM3)):
    if not (YM3.iloc[i]+1>=res[i] and YM3.iloc[i]-1<=res[i]):
        cls2[i]=1
print(classification_report(cls2,cls))

In [ ]:
from sklearn.metrics import classification_report
np.count_nonzero(np.round(model2.predict(XM2))==0)

143/177 delay flights at M2 model.

In [ ]:
import skopt
from skopt.utils import use_named_args
from skopt import gp_minimize

space=[]

In [ ]:
model3.params["objective"] = "regression"
explainer1 = shap.TreeExplainer(model1)

In [ ]:
explainer3 = shap.TreeExplainer(model3)
DEP=0

In [ ]:
space=[]
space.append(skopt.space.space.Integer(-20, 40, name='dep_delay'))
import warnings
warnings.filterwarnings('ignore', message='The objective has been evaluated at this point before.')

In [ ]:
# define the function used to evaluate a given configuration
@use_named_args(space)
def evaluate_model(**params):
    global cnt,x_t,x_t2,t
    print("Iter",cnt,"Dep_Del",params['dep_delay'])
    cnt=cnt+1
    x_temp=x_t.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp2=x_t2.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp.loc[1,'dep_del']=params['dep_delay']
    x_temp.loc[1,'act_dep_m']=x_temp.loc[1,'sch_dep_m']+x_temp.loc[1,'dep_del']
    value1 = explainer1.shap_values(x_temp)
    pred=value1.sum(axis=1)+explainer1.expected_value
    #Here pred is taxi_out
    x_temp2['taxi']=pred
    x_temp2['dep_del']=x_temp['dep_del']
    x_temp2['act_dep_m']=x_temp['act_dep_m']
    x_temp2['wh_off']=x_temp2['act_dep_m']+x_temp2['taxi']
    x_temp2['wh_on']=x_temp2['act_dep_m']+x_temp2['sch_elp']-13    
    #print(x_temp)
    value2 = explainer3.shap_values(x_temp2)
    sum_t=[i[1].sum() for i in value2]
    temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
    prob=temp_t/np.sum(temp_t)
    t=min(np.argmax(prob),t)
    print(prob,t)
    # new loss if want to atleast maximize the probability of one class "metric" i.e. mainitaining this class
    # return 1-prob[t]
    # reutn 1-prob[0] to maximize prob of that class
    # current to shift distributoin towards left side.
    # weighted probability does not give minimum
    return np.sum([(t-i+1)*(1-prob[i]) for i in range(0,t+1)])+np.sum([(i+1)*(prob[i]) for i in range(t+1,6)])#+0.01*(params['dep_delay']+20)


In [ ]:
from skopt.space import Integer
space=[]
space.append(Integer(-20, 30, name='dep_delay'))
z=15
cnt=1
k=ind[z] #limit is [0-176]
x_t=XM1.iloc[k:k+2,:].copy(deep=True)
x_t.iloc[1,:]=XM1.iloc[k,:]
print(x_t.iloc[0,:])
print("out",model1.predict(x_t.iloc[0,:]))
print("Actual",YM1.iloc[k])
print(model2.predict(XM2.iloc[k,:]))
x_t2=XM3.iloc[z:z+2,:].copy(deep=True)
x_t2.iloc[1,:]=XM3.iloc[z,:]
value2 = explainer3.shap_values(x_t2)
sum_t=[i[0].sum() for i in value2]
#print(sum_t)
temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
prob=temp_t/np.sum(temp_t)
t=np.argmax(prob)
print(prob)
print(t)
print(YM3.iloc[z])

t=np.argmax(prob)
result = gp_minimize(evaluate_model,space,n_initial_points=5,n_calls=20)
print('Best dep_delay: %s' % (result.x))
params={(result.x)[0]}
print(prob)
res=evaluate_model(params)
from skopt.plots import plot_convergence

plot_convergence(result)
from skopt.plots import plot_objective
plot_objective(result, n_points=20)

In [ ]:
# define the function used to evaluate a given configuration
@use_named_args(space)
def evaluate_model2(**params):
    global cnt,x_t,x_t2,t
    print("Iter",cnt,"Dep_Del",params['dep_delay'])
    cnt=cnt+1
    x_temp=x_t.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp2=x_t2.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp.loc[1,'dep_del']=x_temp.loc[1,'dep_del']+params['dep_delay']
    x_temp.loc[1,'act_dep_m']=x_temp.loc[1,'sch_dep_m']+x_temp.loc[1,'dep_del']
    value1 = explainer1.shap_values(x_temp)
    pred=value1.sum(axis=1)+explainer1.expected_value
    #Here pred is taxi_out
    x_temp2['taxi']=pred
    x_temp2['dep_del']=x_temp['dep_del']
    x_temp2['act_dep_m']=x_temp['act_dep_m']
    x_temp2['wh_off']=x_temp2['act_dep_m']+x_temp2['taxi']
    x_temp2['wh_on']=x_temp2['act_dep_m']+x_temp2['sch_elp']-13    
    #print(x_temp)
    value2 = explainer3.shap_values(x_temp2)
    sum_t=[i[1].sum() for i in value2]
    temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
    prob2=temp_t/np.sum(temp_t)
    t=min(np.argmax(prob2),t)
    print(prob2,t)
    # new loss if want to atleast maximize the probability of one class "metric" i.e. mainitaining this class
    # return 1-prob[t]
    # reutn 1-prob[0] to maximize prob of that class
    # current to shift distributoin towards left side.
    # weighted probability does not give minimum
    return np.sum([(t-i+1)*(1-prob2[i]) for i in range(0,t+1)])+np.sum([(i+1)*(prob2[i]) for i in range(t+1,6)])+0.005*(abs(params['dep_delay']))


In [ ]:
from skopt.space import Integer
space=[]
space.append(Integer(-20, 20, name='dep_delay'))
cnt=1
k=ind[z] #limit is [0-176]
DEP=XM1.iloc[k,5]
x_t=XM1.iloc[k:k+2,:].copy(deep=True)
x_t.iloc[1,:]=XM1.iloc[k,:]
print(x_t.iloc[0,:])
print("out",model1.predict(x_t.iloc[0,:]))
print("Actual",YM1.iloc[k])
print(model2.predict(XM2.iloc[k,:]))
x_t2=XM3.iloc[z:z+2,:].copy(deep=True)
x_t2.iloc[1,:]=XM3.iloc[z,:]
value2 = explainer3.shap_values(x_t2)
sum_t=[i[0].sum() for i in value2]
#print(sum_t)
temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
prob=temp_t/np.sum(temp_t)
t=np.argmax(prob)
print(prob)
print(t)
print(YM3.iloc[z])

t=np.argmax(prob)
result = gp_minimize(evaluate_model2,space,n_initial_points=5,n_calls=20)
print('Best dep_delay: %s' % (result.x))
params={(result.x)[0]}
print(prob)
res=evaluate_model2(params)
from skopt.plots import plot_convergence

plot_convergence(result)
from skopt.plots import plot_objective
plot_objective(result, n_points=20)

In [ ]:
# define the function used to evaluate a given configuration
@use_named_args(space)
def evaluate_model22(**params):
    global cnt,x_t,x_t2,t
    print("Iter",cnt,"Dep_Del",params['dep_delay'])
    cnt=cnt+1
    x_temp=x_t.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp2=x_t2.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp.loc[1,'dep_del']=params['dep_delay']
    x_temp.loc[1,'act_dep_m']=x_temp.loc[1,'sch_dep_m']+x_temp.loc[1,'dep_del']
    value1 = explainer1.shap_values(x_temp)
    pred=value1.sum(axis=1)+explainer1.expected_value
    #Here pred is taxi_out
    x_temp2['taxi']=pred
    x_temp2['dep_del']=x_temp['dep_del']
    x_temp2['act_dep_m']=x_temp['act_dep_m']
    x_temp2['wh_off']=x_temp2['act_dep_m']+x_temp2['taxi']
    x_temp2['wh_on']=x_temp2['act_dep_m']+x_temp2['sch_elp']-13    
    #print(x_temp)
    value2 = explainer3.shap_values(x_temp2)
    sum_t=[i[1].sum() for i in value2]
    temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
    prob2=temp_t/np.sum(temp_t)
    t=min(np.argmax(prob2),t)
    print(prob2,t)
    # new loss if want to atleast maximize the probability of one class "metric" i.e. mainitaining this class
    # return 1-prob[t]
    # reutn 1-prob[0] to maximize prob of that class
    # current to shift distributoin towards left side.
    # weighted probability does not give minimum
    return np.sum([(t-i+1)*(1-prob2[i]) for i in range(0,t+1)])+np.sum([(i+1)*(prob2[i]) for i in range(t+1,6)])+0.005*(abs(params['dep_delay']))


In [ ]:
from skopt.space import Integer
space=[]
space.append(Integer(-20, 30, name='dep_delay'))
cnt=1
k=ind[z] #limit is [0-176]
x_t=XM1.iloc[k:k+2,:].copy(deep=True)
x_t.iloc[1,:]=XM1.iloc[k,:]
print(x_t.iloc[0,:])
print("out",model1.predict(x_t.iloc[0,:]))
print("Actual",YM1.iloc[k])
print(model2.predict(XM2.iloc[k,:]))
x_t2=XM3.iloc[z:z+2,:].copy(deep=True)
x_t2.iloc[1,:]=XM3.iloc[z,:]
value2 = explainer3.shap_values(x_t2)
sum_t=[i[0].sum() for i in value2]
#print(sum_t)
temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
prob=temp_t/np.sum(temp_t)
t=np.argmax(prob)
print(prob)
print(t)
print(YM3.iloc[z])

t=np.argmax(prob)
result = gp_minimize(evaluate_model22,space,n_initial_points=5,n_calls=20)
print('Best dep_delay: %s' % (result.x))
params={(result.x)[0]}
print(prob)
res=evaluate_model2(params)
from skopt.plots import plot_convergence

plot_convergence(result)
from skopt.plots import plot_objective
plot_objective(result, n_points=20)

In [ ]:
# define the function used to evaluate a given configuration
@use_named_args(space)
def evaluate_model3(**params):
    global cnt,x_t,x_t2,tp
    #print("Iter",cnt,"Dep_Del",params['dep_delay'])
    cnt=cnt+1
    x_temp=x_t.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp2=x_t2.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp.loc[1,'dep_del']=params['dep_delay']
    x_temp.loc[1,'act_dep_m']=x_temp.loc[1,'sch_dep_m']+x_temp.loc[1,'dep_del']
    value1 = explainer1.shap_values(x_temp)
    pred=value1.sum(axis=1)+explainer1.expected_value
    #Here pred is taxi_out
    x_temp2['taxi']=pred
    x_temp2['dep_del']=x_temp['dep_del']
    x_temp2['act_dep_m']=x_temp['act_dep_m']
    x_temp2['wh_off']=x_temp2['act_dep_m']+x_temp2['taxi']
    x_temp2['wh_on']=x_temp2['act_dep_m']+x_temp2['sch_elp']-13    
    #print(x_temp)
    value2 = explainer3.shap_values(x_temp2)
    sum_t=[i[1].sum() for i in value2]
    temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
    prob3=temp_t/np.sum(temp_t)
    #t=min(np.argmax(prob3),t)
   # print(prob3,tp)
    # new loss if want to atleast maximize the probability of one class "metric" i.e. mainitaining this class
    # return 1-prob[t]
    # reutn 1-prob[0] to maximize prob of that class
    # current to shift distributoin towards left side.
    # weighted probability does not give minimum
    return 1-prob3[tp]

In [ ]:
t=np.argmax(prob)
print(prob,t)
for tp in range(0,t+1):
    result = gp_minimize(evaluate_model3,space,n_initial_points=5,n_calls=20)
    print('\nBest dep_delay: %s' % (result.x))
    params={(result.x)[0]}
    print(evaluate_model3(params),tp)
    #plot_convergence(result)
    plot_objective(result, n_points=20)

In [ ]:
def eval_point(dep_del):
    global x_t,x_t2
    #print("Iter",cnt,"Dep_Del",params['dep_delay'])
    x_temp=x_t.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp2=x_t2.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp.loc[1,'dep_del']=dep_del
    x_temp.loc[1,'act_dep_m']=x_temp.loc[1,'sch_dep_m']+x_temp.loc[1,'dep_del']
    value1 = explainer1.shap_values(x_temp)
    pred=value1.sum(axis=1)+explainer1.expected_value
    #Here pred is taxi_out
    x_temp2['taxi']=pred
    x_temp2['dep_del']=x_temp['dep_del']
    x_temp2['act_dep_m']=x_temp['act_dep_m']
    x_temp2['wh_off']=x_temp2['act_dep_m']+x_temp2['taxi']
    x_temp2['wh_on']=x_temp2['act_dep_m']+x_temp2['sch_elp']-13    
    #print(x_temp)
    value2 = explainer3.shap_values(x_temp2)
    sum_t=[i[1].sum() for i in value2]
    temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
    prob=temp_t/np.sum(temp_t)
    tp=np.argmax(prob)
    #rint(prob,t)
    # new loss if want to atleast maximize the probability of one class "metric" i.e. mainitaining this class
    # return 1-prob[t]
    # reutn 1-prob[0] to maximize prob of that class
    # current to shift distributoin towards left side.
    # weighted probability does not give minimum
    #print(pred[1])
    return pred[1],tp

In [ ]:
# define the function used to evaluate a given configuration
@use_named_args(space)
def evaluate_model4(**params):
    global x_t,x_t2,t
    #print("Iter",cnt,"Dep_Del",params['dep_delay'])
    x_temp=x_t.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp2=x_t2.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp.loc[1,'dep_del']=params['dep_delay']
    x_temp.loc[1,'act_dep_m']=x_temp.loc[1,'sch_dep_m']+x_temp.loc[1,'dep_del']
    value1 = explainer1.shap_values(x_temp)
    pred=value1.sum(axis=1)+explainer1.expected_value
    #Here pred is taxi_out
    x_temp2['taxi']=pred
    x_temp2['dep_del']=x_temp['dep_del']
    x_temp2['act_dep_m']=x_temp['act_dep_m']
    x_temp2['wh_off']=x_temp2['act_dep_m']+x_temp2['taxi']
    x_temp2['wh_on']=x_temp2['act_dep_m']+x_temp2['sch_elp']-13    
    #print(x_temp)
    value2 = explainer3.shap_values(x_temp2)
    sum_t=[i[1].sum() for i in value2]
    temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
    prob=temp_t/np.sum(temp_t)
    t=min(np.argmax(prob),t)
    #rint(prob,t)
    # new loss if want to atleast maximize the probability of one class "metric" i.e. mainitaining this class
    # return 1-prob[t]
    # reutn 1-prob[0] to maximize prob of that class
    # current to shift distributoin towards left side.
    # weighted probability does not give minimum
    return np.sum([(t-i+1)*(1-prob[i]) for i in range(0,t+1)])+np.sum([(i+1)*(prob[i]) for i in range(t+1,6)])#+0.01*(params['dep_delay']+20)

In [ ]:
from skopt.space import Integer
time_list=[]
import csv
f=open("temp.csv","w+", newline="")
writer = csv.writer(f)
from tqdm import tqdm
for z in tqdm(range(0,len(ind)-1)):
    ls=[[]]
    k=ind[z] #limit is [0-176]
    x_t=XM1.iloc[k:k+2,:].copy(deep=True)
    x_t.iloc[1,:]=XM1.iloc[k,:]
    #print(x_t)
    ls[0].append(x_t.iloc[0,5])
    ls[0].append(YM1.iloc[k])
    ls[0].append(YM3.iloc[z])
    x_t2=XM3.iloc[z:z+2,:].copy(deep=True)
    x_t2.iloc[1,:]=XM3.iloc[z,:]
    value2 = explainer3.shap_values(x_t2)
    sum_t=[i[0].sum() for i in value2]
    #print(sum_t)
    temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
    prob=temp_t/np.sum(temp_t)
    tp=np.argmax(prob)
    ls.append([x_t.iloc[0,5],model1.predict(XM1.iloc[k]),tp])
    for num in [15,20,30,60]:    
        space=[]
        space.append(Integer(-20, 40, name='dep_delay'))
        t=tp
        result = gp_minimize(evaluate_model4,space,n_initial_points=5,n_calls=num)
        tax,cl=eval_point((result.x)[0])
        ls.append([(result.x)[0],tax,cl])
    time_list.append(ls)
    writer.writerows(ls)
    print(len(time_list),len(ls),len(ls[-1]))

In [ ]:
import csv

f=open("temp_2.csv","w+")
writer = csv.writer(f)
for i in time_list:
    writer.writerows(i)

In [ ]:
# define the function used to evaluate a given configuration
@use_named_args(space)
def evaluate_model51(**params):
    global x_t,x_t2,t
    #print("Iter",cnt,"Dep_Del",params['dep_delay'])
    x_temp=x_t.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp2=x_t2.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp.loc[1,'dep_del']=params['dep_delay']+x_temp.loc[1,'dep_del']
    x_temp.loc[1,'act_dep_m']=x_temp.loc[1,'sch_dep_m']+x_temp.loc[1,'dep_del']
    value1 = explainer1.shap_values(x_temp)
    pred=value1.sum(axis=1)+explainer1.expected_value
    #Here pred is taxi_out
    x_temp2['taxi']=pred
    x_temp2['dep_del']=x_temp['dep_del']
    x_temp2['act_dep_m']=x_temp['act_dep_m']
    x_temp2['wh_off']=x_temp2['act_dep_m']+x_temp2['taxi']
    x_temp2['wh_on']=x_temp2['act_dep_m']+x_temp2['sch_elp']-13    
    #print(x_temp)
    value2 = explainer3.shap_values(x_temp2)
    sum_t=[i[1].sum() for i in value2]
    temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
    prob=temp_t/np.sum(temp_t)
    t=min(np.argmax(prob),t)
    #rint(prob,t)
    # new loss if want to atleast maximize the probability of one class "metric" i.e. mainitaining this class
    # return 1-prob[t]
    # reutn 1-prob[0] to maximize prob of that class
    # current to shift distributoin towards left side.
    # weighted probability does not give minimum
    return np.sum([(t-i+1)*(1-prob[i]) for i in range(0,t+1)])+np.sum([(i+1)*(prob[i]) for i in range(t+1,6)])+0.005*(abs(params['dep_delay']))

In [ ]:
# define the function used to evaluate a given configuration
@use_named_args(space)
def evaluate_model52(**params):
    global x_t,x_t2,t
    #print("Iter",cnt,"Dep_Del",params['dep_delay'])
    x_temp=x_t.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp2=x_t2.copy(deep=True).reset_index().drop(['index'],axis=1)
    x_temp.loc[1,'dep_del']=params['dep_delay']
    x_temp.loc[1,'act_dep_m']=x_temp.loc[1,'sch_dep_m']+x_temp.loc[1,'dep_del']
    value1 = explainer1.shap_values(x_temp)
    pred=value1.sum(axis=1)+explainer1.expected_value
    #Here pred is taxi_out
    x_temp2['taxi']=pred
    x_temp2['dep_del']=x_temp['dep_del']
    x_temp2['act_dep_m']=x_temp['act_dep_m']
    x_temp2['wh_off']=x_temp2['act_dep_m']+x_temp2['taxi']
    x_temp2['wh_on']=x_temp2['act_dep_m']+x_temp2['sch_elp']-13    
    #print(x_temp)
    value2 = explainer3.shap_values(x_temp2)
    sum_t=[i[1].sum() for i in value2]
    temp_t=[1/(1+np.exp(-sum_t[i]-explainer3.expected_value[i])) for i in range(0,6)]
    prob=temp_t/np.sum(temp_t)
    t=min(np.argmax(prob),t)
    #rint(prob,t)
    # new loss if want to atleast maximize the probability of one class "metric" i.e. mainitaining this class
    # return 1-prob[t]
    # reutn 1-prob[0] to maximize prob of that class
    # current to shift distributoin towards left side.
    # weighted probability does not give minimum
    return np.sum([(t-i+1)*(1-prob[i]) for i in range(0,t+1)])+np.sum([(i+1)*(prob[i]) for i in range(t+1,6)])+0.005*(abs(params['dep_delay']))

In [ ]:
from skopt.space import Integer
time_list2=[]
from tqdm import tqdm
for z in tqdm(range(0,len(ind)-1)):
    ls=[[]]
    k=ind.iloc[z] #limit is [0-176]
    x_t=XM1.iloc[k:k+2,:].copy(deep=True)
    x_t.iloc[1,:]=XM1.iloc[k,:]
    #print(x_t)
    ls[0].append(x_t.iloc[0,5])
    ls[0].append(YM1.iloc[k])
    ls[0].append(YM3.iloc[z])
    
    space=[]
    space.append(Integer(-20, 20, name='dep_delay'))
    t=tp
    result = gp_minimize(evaluate_model51,space,n_initial_points=5,n_calls=20)
    tax,cl=eval_point((result.x)[0]+x_t.iloc[1,'dep_del'])
    ls.append([(result.x)[0],tax,cl])
    
    space=[]
    space.append(Integer(-20, 40, name='dep_delay'))
    t=tp
    result = gp_minimize(evaluate_model52,space,n_initial_points=5,n_calls=20)
    tax,cl=eval_point((result.x)[0])
    ls.append([(result.x)[0],tax,cl])
    
    
    time_list2.append(ls)
len(time_list2)

In [ ]:
import csv

f=open("temp2.csv","w+", newline="")
writer = csv.writer(f)
for i in time_list2:
    writer.writerows(i)

In [ ]:
l1=pd.read_csv("temp.csv",header=None)
l2=pd.read_csv("temp2.csv",header=None)

In [ ]:
x=[0,1,2,3,4,5]
y=['ori','15 min','20 min','30 min','60 min','pred']
z=np.zeros(6,6)
for i in range(0,len(l1),5):
    z[l1.iloc[i,2]][0]+=1
    z[l1.iloc[i+1,2]][1]+=1
    z[l1.iloc[i+2,2]][2]+=1
    z[l1.iloc[i+3,2]][3]+=1
    z[l1.iloc[i+4,2]][4]+=1
    z[np.argmax[model3.predict()]][5]+=1
    

In [ ]:
import matplotlib.pyplot as plt
 
 
# defining surface and axes
x = np.outer(np.linspace(-2, 2, 10), np.ones(1))
print(x)
y = x.copy().T
z = np.cos(x ** 2 + y ** 3)
print(x.shape,y.shape,z.shape)
fig = plt.figure()
 
# syntax for 3-D plotting
ax = plt.axes(projection ='3d')
 
# syntax for plotting
ax.plot_surface(x, y, z, cmap ='viridis', edgecolor ='green')
ax.set_title('Surface plot geeks for geeks')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# setup the figure and axes
fig = plt.figure(figsize=(8, 3))
ax1 = fig.add_subplot(121, projection='3d')

# fake data
_x = np.arange(4)
_y = np.arange(5)
_xx, _yy = np.meshgrid(_x, _y)
x, y = _xx.ravel(), _yy.ravel()

top = x + y
bottom = np.zeros_like(top)
width = depth = 1

ax1.bar3d(x, y, bottom, width, depth, top, shade=True)
ax1.set_title('Shaded')


plt.show()

In [ ]:
import numpy as np
x = np.linspace(-6, 6, 30)
y = np.linspace(-6, 6, 30)

def z_function(x, y):
    return np.sin(np.sqrt(x ** 2 + y ** 2))
X, Y = np.meshgrid(x, y)
Z = z_function(X, Y)



import matplotlib.pyplot as plt
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.plot_wireframe(X, Y, Z, color='green')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

In [ ]:
from matplotlib import cm
# Normalize the colors based on Z value
norm = plt.Normalize(Z2.min(), Z2.max())
colors = cm.jet(norm(Z2))
ax = plt.axes(projection='3d')
surf = ax.plot_surface(X2, Y2, Z2, facecolors=colors, shade=False)
surf.set_facecolor((0,0,0,0))